# Melanoma Classification

In [45]:
import cv2
import os
import pickle

from sklearn.metrics import precision_recall_curve
from keras.utils import to_categorical
from keras.models import Sequential
import keras
from keras.preprocessing.image import img_to_array
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
from keras import backend as K
import random
from sklearn.cross_validation import train_test_split
from sklearn.utils import class_weight

In [2]:
if not os.path.exists('pickled'):
    os.makedirs('pickled')

# Data Pre-Processing



In [3]:
#reading melanoma data
posimages = os.listdir("melanoma/")
data = []
labels = []
if not os.path.exists('pickled/data.pkl'):
    for file in posimages:
        image = cv2.imread("melanoma/" + file)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        data.append(image)
        labels.append(1)

    negimages = os.listdir("others/")
    for file in negimages:
        image = cv2.imread("others/" + file)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        data.append(image)
        labels.append(0)
        
    d = {}
    d['data'] = data
    d['labels'] = labels
    pickle.dump(d, open('pickled/data.pkl', 'wb'))
else:
    d = pickle.load(open( 'pickled/data.pkl', 'rb'))
    data = d['data']
    labels = d['labels']

In [11]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

(x_train, x_test, y_train, y_test) = train_test_split(data,labels, test_size=0.40, random_state=42)
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

# Architecture Of the Model

LAYER 1
A 2D CNN layer with activation function of RelU 32 neurons, kernel size 5X5. Maxpooling and strides are taken for downsampling.
LAYER 2
A 2D CNN Layer with activation function sigmoid and 64 neurons.
LAYER 3
Dropout layer to prevent overfitting
LAYER 4
A dense layer with 64 neurons
LAYER 5
The final layer with softmax activation function and 2 neurons for classification.


In [12]:
batch_size = 64
epochs = 10

img_rows, img_cols = 64, 64

input_shape = (img_rows, img_cols,3)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5),strides = (2,2), activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

x_train shape: (1200, 64, 64, 3)
1200 train samples
800 test samples


In [49]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1score(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2*r*p/(r+p)

In [60]:
#compiling model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', precision, recall, f1score])

In [62]:
#training and testing model
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test), class_weight = class_weights)

Train on 1200 samples, validate on 800 samples
Epoch 1/10
1200/1200 [==============================] - 9s 8ms/step - loss: 0.6335 - acc: 0.5192 - precision: 0.5192 - recall: 0.5192 - f1score: 0.5192 - val_loss: 0.6943 - val_acc: 0.1900 - val_precision: 0.5000 - val_recall: 1.0000 - val_f1score: 0.6667
Epoch 2/10
1200/1200 [==============================] - 9s 7ms/step - loss: 0.6345 - acc: 0.4725 - precision: 0.5797 - recall: 0.8550 - f1score: 0.6887 - val_loss: 0.6910 - val_acc: 0.8100 - val_precision: 0.5000 - val_recall: 1.0000 - val_f1score: 0.6667
Epoch 3/10
1200/1200 [==============================] - 8s 7ms/step - loss: 0.6332 - acc: 0.5050 - precision: 0.6067 - recall: 0.9175 - f1score: 0.7300 - val_loss: 0.6891 - val_acc: 0.8100 - val_precision: 0.5000 - val_recall: 1.0000 - val_f1score: 0.6667
Epoch 4/10
1200/1200 [==============================] - 9s 8ms/step - loss: 0.6338 - acc: 0.4700 - precision: 0.5938 - recall: 0.9092 - f1score: 0.7180 - val_loss: 0.6865 - val_acc: 0.8

# Accuracy Metrics

In [63]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test F1 score:', score[4])

Test loss: 0.674210689068
Test F1 score: 0.81
